<a href="https://colab.research.google.com/github/muhammadali74/MLAlgorithms/blob/master/Assignment_1_sn07590.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 335: Introduction to Large Language Models
## Assignment 01
### **Total Marks**: 100
### **Deadline**: Sunday, 3rd March, 2024, 11:59 PM
### **Name**: Syed Muhammad Ali Naqvi
### **ID**: sn07590

#Instructions

1. Please rename your notebook as *Assignment_1_aa1234.ipynb* before the final submission. Notebooks which do not follow appropriate naming convention will not be graded.

2. Please submit your own work. If you have any questions, please feel free to reach out to the course instructors or RA.



# Assignment Overview

In this assignment, you are required to fine tune a LLM model of your that classifies which human value category a textual arguement belongs to. Your model will evaluated against 1-baseline, random-baseline results on the following dataset: test, Nahjalbalagha, Zhihu


# Setup



In [ ]:
# IMPORT ALL YOUR LIBRARIES
# SUGGESTED LIBRARIES
!pip install accelerate -U

import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, AutoModelForSequenceClassification, TrainingArguments, EvalPrediction
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.6 MB/s eta 0:00:00


# Download Files


##Evaluator

In [ ]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/evaluator/evaluator.py

--2024-03-01 15:36:34--  https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/evaluator/evaluator.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8708 (8.5K) [text/plain]
Saving to: ‘evaluator.py’

evaluator.py        100%[===================>]   8.50K  --.-KB/s    in 0s      

2024-03-01 15:36:34 (89.3 MB/s) - ‘evaluator.py’ saved [8708/8708]



## 1-Baseline

In [ ]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/1-baseline/1-baseline.py

--2024-03-01 15:36:34--  https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/1-baseline/1-baseline.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3613 (3.5K) [text/plain]
Saving to: ‘1-baseline.py’

1-baseline.py       100%[===================>]   3.53K  --.-KB/s    in 0s      

2024-03-01 15:36:35 (56.9 MB/s) - ‘1-baseline.py’ saved [3613/3613]



## Random-Baseline

In [ ]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/random-baseline/random-baseline.py

--2024-03-01 15:36:35--  https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/random-baseline/random-baseline.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5442 (5.3K) [text/plain]
Saving to: ‘random-baseline.py’

random-baseline.py  100%[===================>]   5.31K  --.-KB/s    in 0s      

2024-03-01 15:36:35 (52.1 MB/s) - ‘random-baseline.py’ saved [5442/5442]



## Dataset Files

In [ ]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://zenodo.org/api/records/10564870/files-archive

--2024-03-01 15:36:35--  https://zenodo.org/api/records/10564870/files-archive
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.103.159, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘files-archive’

files-archive           [              <=>   ]  54.70M  17.4MB/s    in 3.1s    

2024-03-01 15:36:39 (17.4 MB/s) - ‘files-archive’ saved [57353430]



In [ ]:
# DO NOT EDIT
# RUN ONLY ONCE
!unzip /content/files-archive -d Dataset

Archive:  /content/files-archive
 extracting: Dataset/value-categories.json  
 extracting: Dataset/arguments-validation-zhihu.tsv  
 extracting: Dataset/level1-labels-training.tsv  
 extracting: Dataset/arguments-test-nahjalbalagha.tsv  
 extracting: Dataset/arguments-test.tsv  
 extracting: Dataset/arguments-training.tsv  
 extracting: Dataset/arguments-validation.tsv  
 extracting: Dataset/labels-test-nahjalbalagha.tsv  
 extracting: Dataset/labels-test-nyt.tsv  
 extracting: Dataset/labels-test.tsv  
 extracting: Dataset/labels-training.tsv  
 extracting: Dataset/level1-labels-test-nahjalbalagha.tsv  
 extracting: Dataset/level1-labels-test.tsv  
 extracting: Dataset/level1-labels-test-nyt.tsv  
 extracting: Dataset/labels-validation.tsv  
 extracting: Dataset/labels-validation-zhihu.tsv  
 extracting: Dataset/meta-arguments-g.tsv  
 extracting: Dataset/meta-arguments-c.tsv  
 extracting: Dataset/meta-arguments-e.tsv  
 extracting: Dataset/meta-arguments-f.tsv  
 extracting: Dataset

In [ ]:
!mkdir /content/Dataset/zhihu
!mkdir /content/Dataset/nahjalbalagha
!mkdir /content/Dataset/train
!mkdir /content/Dataset/test
!mkdir /content/Dataset/validation

In [ ]:
!mv /content/Dataset/*-zhihu.tsv /content/Dataset/zhihu
!mv /content/Dataset/*-nahjalbalagha.tsv /content/Dataset/nahjalbalagha
!mv /content/Dataset/*-nahjalbalagha.tsv /content/Dataset/nahjalbalagha
!mv /content/Dataset/*-training.tsv /content/Dataset/train
!mv /content/Dataset/*-test.tsv /content/Dataset/test
!mv /content/Dataset/*-validation.tsv /content/Dataset/validation

mv: cannot stat '/content/Dataset/*-nahjalbalagha.tsv': No such file or directory


# Background Information

## Human Value Detection 2023 <br/>
## SemEval 2023 Task 4. ValueEval: Identification of Human Values behind Arguments



Given a textual argument and a human value category, classify whether or not the argument draws on that category. This task uses a set of 20 value categories compiled from the social science literature and described in our [ACL paper](https://webis.de/publications.html#kiesel_2022b). Arguments are given as premise text, conclusion text, and binary stance of the premise to the conclusion ("in favor of" or "against").

The 20 value categories are shown here on Schwartz' value continuum below:

[![JEPBxUu.md.png](https://iili.io/JEPBxUu.md.png)](https://freeimage.host/i/JEPBxUu)







## Data


Data is provided as tab-separated values files with one header line. The arguments-validation.tsv files contain one argument per line: its unique argument ID, the conclusion, the premise's stance towards the conclusion, and the premise itself. Example with tab-separated columns are shown below

<pre><span class="column">Argument ID</span>	<span class="column">Conclusion</span>	<span class="column">Stance</span>	<span class="column">Premise</span>
<span class="column">A01010</span>	<span class="column">We should prohibit school prayer</span>	<span class="column">against</span>	<span class="column">it should be allowed if the student wants to pray as long as it is not interfering with his classes</span>
<span class="column">A01011</span>	<span class="column">We should abolish the three-strikes laws</span>	<span class="column">in favor of</span>	<span class="column">three strike laws can cause young people to be put away for life without a chance to straight out their life</span>
<span class="column">A01012</span>	<span class="column">The use of public defenders should be mandatory</span>	<span class="column">in favor of</span>	<span class="column">the use of public defenders should be mandatory because some people don't have money for a lawyer and this would help those that don't</span>
</pre>

The labels-validation.tsv  files also contain one argument per line: its unique argument ID and one column for each of the 20 value categories with a 1 meaning that the argument resorts to the value category and a 0 that not. Example with tab-separated columns are shown below:

<pre><span class="column">Argument ID</span>	<span class="column">Self-direction: thought</span>	<span class="column">Self-direction: action</span>	<span class="column">Stimulation</span>	<span class="column">Hedonism</span>	<span class="column">Achievement</span>	<span class="column">Power: dominance</span>	<span class="column">Power: resources</span>	<span class="column">Face</span>	<span class="column">Security: personal</span>	<span class="column">Security: societal</span>	<span class="column">Tradition</span>	<span class="column">Conformity: rules</span>	<span class="column">Conformity: interpersonal</span>	<span class="column">Humility</span>	<span class="column">Benevolence: caring</span>	<span class="column">Benevolence: dependability</span>	<span class="column">Universalism: concern</span>	<span class="column">Universalism: nature</span>	<span class="column">Universalism: tolerance</span>	<span class="column">Universalism: objectivity</span>
<span class="column">A01010</span>	<span class="column">1</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>
<span class="column">A01011</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">1</span>
<span class="column">A01012</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>
</pre>

In addition, there are other datasets for evaluating the robustness of our model: validation-zhihu from the recommendation and hotlist section of the Chinese question-answering website Zhihu, test-nahjalbalagha from and based on the Nahj al-Balagha.



## Evaluation

Runs are evaluated on the basis of F1-score, Precision, and Recall: averaged over all value categories and for each category individually.

## Baseline Results

In [ ]:
# DO NOT EDIT
# RUN ONLY ONCE
!mkdir /content/baseline
!mkdir /content/output

### 1-Baseline

#### Test Dataset

In [ ]:
# DO NOT EDIT
!python3 /content/1-baseline.py --inputDataset /content/Dataset/test --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/test/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/test/arguments-test.tsv
Labeling 1576 instances
Detected values: {'Power: dominance', 'Universalism: objectivity', 'Universalism: concern', 'Security: societal', 'Tradition', 'Conformity: interpersonal', 'Conformity: rules', 'Hedonism', 'Benevolence: dependability', 'Security: personal', 'Self-direction: thought', 'Power: resources', 'Universalism: nature', 'Humility', 'Face', 'Self-direction: action', 'Universalism: tolerance', 'Stimulation', 'Achievement', 'Benevolence: caring'}
Writing run file
Reading /content/Dataset/test/labels-test.tsv
Reading /content/baseline/run.tsv
Truth labels: 1576
Run labels:   1576
measure {
 key: "F1"
 value: "0.26293020308065357"
}
measure {
 key: "Precision"
 value: "0.15136421319796953"
}
measure {
 key: "Recall"
 value: "1.0"
}


#### Zhihu

In [ ]:
# DO NOT EDIT
!python3 /content/1-baseline.py --inputDataset /content/Dataset/zhihu/ --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/zhihu/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/zhihu/arguments-validation-zhihu.tsv
Labeling 100 instances
Detected values: {'Conformity: interpersonal', 'Tradition', 'Universalism: nature', 'Humility', 'Stimulation', 'Self-direction: thought', 'Benevolence: dependability', 'Power: resources', 'Benevolence: caring', 'Self-direction: action', 'Hedonism', 'Conformity: rules', 'Security: personal', 'Universalism: objectivity', 'Achievement', 'Universalism: concern', 'Power: dominance', 'Face', 'Security: societal', 'Universalism: tolerance'}
Writing run file
Reading /content/Dataset/zhihu/labels-validation-zhihu.tsv
Reading /content/baseline/run.tsv
Truth labels: 100
Run labels:   100
measure {
 key: "F1"
 value: "0.2292179045745204"
}
measure {
 key: "Precision"
 value: "0.12944444444444445"
}
measure {
 key: "Recall"
 value: "1.0"
}


#### Nahjalbalagha

In [ ]:
# DO NOT EDIT
!python3 /content/1-baseline.py --inputDataset /content/Dataset/nahjalbalagha/ --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/nahjalbalagha/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/nahjalbalagha/arguments-test-nahjalbalagha.tsv
Labeling 279 instances
Detected values: {'Power: resources', 'Self-direction: action', 'Benevolence: caring', 'Conformity: interpersonal', 'Tradition', 'Humility', 'Achievement', 'Power: dominance', 'Universalism: nature', 'Stimulation', 'Security: personal', 'Face', 'Universalism: concern', 'Self-direction: thought', 'Benevolence: dependability', 'Security: societal', 'Universalism: tolerance', 'Hedonism', 'Conformity: rules', 'Universalism: objectivity'}
Writing run file
Reading /content/Dataset/nahjalbalagha/labels-test-nahjalbalagha.tsv
Reading /content/baseline/run.tsv
Truth labels: 279
Run labels:   279
measure {
 key: "F1"
 value: "0.12845882944826426"
}
measure {
 key: "Precision"
 value: "0.0686379928315412"
}
measure {
 key: "Recall"
 value: "1.0"
}


### Random-Baseline


#### Test

In [ ]:
# DO NOT EDIT
!python3 /content/random-baseline.py --inputDataset /content/Dataset/test --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/test/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/test/arguments-test.tsv
Labeling 1576 instances
Detected values: {'Hedonism', 'Power: dominance', 'Conformity: rules', 'Security: societal', 'Benevolence: caring', 'Self-direction: action', 'Universalism: objectivity', 'Face', 'Universalism: tolerance', 'Power: resources', 'Security: personal', 'Universalism: nature', 'Self-direction: thought', 'Conformity: interpersonal', 'Stimulation', 'Benevolence: dependability', 'Universalism: concern', 'Achievement', 'Humility', 'Tradition'}
Writing run file
Reading /content/Dataset/test/labels-test.tsv
Reading /content/baseline/run.tsv
Truth labels: 1576
Run labels:   1576
measure {
 key: "F1"
 value: "0.16928213107559772"
}
measure {
 key: "Precision"
 value: "0.16091716724142235"
}
measure {
 key: "Recall"
 value: "0.1785644545857951"
}


#### Zhihu


In [ ]:
# DO NOT EDIT
!python3 /content/random-baseline.py --inputDataset /content/Dataset/zhihu/ --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/zhihu/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/zhihu/arguments-validation-zhihu.tsv
Labeling 100 instances
Detected values: {'Security: personal', 'Achievement', 'Benevolence: dependability', 'Tradition', 'Conformity: rules', 'Benevolence: caring', 'Universalism: objectivity', 'Hedonism', 'Security: societal', 'Universalism: concern', 'Universalism: tolerance', 'Self-direction: action', 'Power: resources', 'Conformity: interpersonal', 'Power: dominance', 'Universalism: nature', 'Humility', 'Stimulation', 'Self-direction: thought', 'Face'}
Writing run file
Reading /content/Dataset/zhihu/labels-validation-zhihu.tsv
Reading /content/baseline/run.tsv
Truth labels: 100
Run labels:   100
measure {
 key: "F1"
 value: "0.1600695451589294"
}
measure {
 key: "Precision"
 value: "0.13721251278185884"
}
measure {
 key: "Recall"
 value: "0.1920637684644476"
}


#### Nahjalbalagha

In [ ]:
# DO NOT EDIT
!python3 /content/random-baseline.py --inputDataset /content/Dataset/nahjalbalagha/ --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/nahjalbalagha/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/nahjalbalagha/arguments-test-nahjalbalagha.tsv
Labeling 279 instances
Detected values: {'Security: societal', 'Universalism: nature', 'Security: personal', 'Universalism: objectivity', 'Face', 'Benevolence: dependability', 'Conformity: interpersonal', 'Tradition', 'Humility', 'Power: resources', 'Conformity: rules', 'Universalism: concern', 'Self-direction: thought', 'Achievement', 'Universalism: tolerance', 'Hedonism', 'Power: dominance', 'Stimulation', 'Self-direction: action', 'Benevolence: caring'}
Writing run file
Reading /content/Dataset/nahjalbalagha/labels-test-nahjalbalagha.tsv
Reading /content/baseline/run.tsv
Truth labels: 279
Run labels:   279
measure {
 key: "F1"
 value: "0.08680905816408373"
}
measure {
 key: "Precision"
 value: "0.06608197570611526"
}
measure {
 key: "Recall"
 value: "0.12648059915572887"
}


# Tasks

## [20 Points] Task 01 - Load Datasets

In this task, you are required to load the Training, Test, Validation, Nahjalbalagha & Zhihu into seperate dataframes.

In [ ]:
# Write your code here
# Training Data
paths = {"training":"training", "test":"test", "validation":"validation", "zhihu":"validation-zhihu", "nahjalbalagha":"test-nahjalbalagha"}
datasets = {"training":{}, "test":{}, "validation":{}, "nahjalbalagha":{}, "zhihu":{}}
subset = ["arguments", "labels", "level1-labels"]


for i in datasets.keys():
  for j in subset:
    if i == "training":
      datasets[i][j] = (pd.read_csv(f"/content/Dataset/train/{j}-{i}.tsv", sep = "\t"))
    else:
      datasets[i][j] = (pd.read_csv(f"/content/Dataset/{i}/{j}-{paths[i]}.tsv", sep = "\t"))

print(datasets['training']['arguments'].shape)
print(datasets['training']['labels'].shape)

# datasets['training']['labels'].head()

(5393, 4)
(5393, 21)


## [10 Points] Task 02 - Define Tokenizer & Model


In this task, you are required to define the Tokenizer and LLM model of your choice.

In [ ]:
# Write your code here
# picking the best model from the leaderboard :)

# from transformers import
# model = "microsoft/phi-1_5"
# tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)

from transformers import AutoTokenizer

model = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model, do_lower_case = True)

rem_cols = [col for col in datasets['training']['labels'].columns[1:]]
id2label = {idx:label for idx, label in enumerate(rem_cols)}
label2id = {label:idx for idx, label in enumerate(rem_cols)}

LLM = AutoModelForSequenceClassification.from_pretrained(model, num_labels=20, problem_type="multi_label_classification", id2label=id2label, label2id=label2id)
print(LLM)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## [20 Points] Task 03 - Optimizer & Hyperparameters


In this task, you are required to define the hyperparameters & the optimizer for training your model.

In [ ]:
# Write your code here

# optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

epochs = 3
# MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16

# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = len(train_df) * epochs)
training_args = TrainingArguments(".", evaluation_strategy="epoch",
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    num_train_epochs=epochs,
)
# training_args

## [20 Points] Task 04 -  Training Loop


In this task, you are required to implement the training loop for fine tuning your model. You are also required to plot on the same graph: Loss vs Epochs & Accuracy vs Epochs

In [ ]:
# Tokeninzing inputs

import numpy as np


# print(rem_cols)
train_labels = datasets['training']['labels'][rem_cols]
train_labels = np.array(train_labels.values.tolist())
train_labels = train_labels.astype(np.float32)
train_args = datasets['training']['arguments']['Premise'].values.tolist()
# print(train_args)
# print(train_labels)
eval_labels = datasets['validation']['labels'][rem_cols]
eval_labels = np.array(eval_labels.values.tolist())
eval_labels = eval_labels.astype(np.float32).tolist()
eval_args = datasets['validation']['arguments']['Premise'].values.tolist()

train_encodings = tokenizer(train_args, padding="max_length", truncation=True, max_length=512)
eval_encodings = tokenizer(eval_args, padding="max_length", truncation=True, max_length=512)



class TextClassifierDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = TextClassifierDataset(train_encodings, train_labels)
eval_dataset = TextClassifierDataset(eval_encodings, eval_labels)

In [ ]:
# Write your code here
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average = 'micro');
    recall = recall_score(y_true, y_pred, average = 'micro');
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy, 'precision': precision, 'recall': recall}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result


trainer = Trainer(model=LLM,args=training_args,train_dataset=train_dataset,eval_dataset=eval_dataset, compute_metrics=compute_metrics)

trainer.train()

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy,Precision,Recall
1,No log,0.351590,0.413478,0.634676,0.059599,0.669483,0.299104
2,0.365600,0.333338,0.471926,0.663113,0.082278,0.689010,0.358861
3,0.285900,0.328580,0.507222,0.683344,0.085970,0.676008,0.405881


TrainOutput(global_step=1014, training_loss=0.3251066499442979, metrics={'train_runtime': 1662.5206, 'train_samples_per_second': 9.732, 'train_steps_per_second': 0.61, 'total_flos': 4257561738203136.0, 'train_loss': 0.3251066499442979, 'epoch': 3.0})

## [10 Points]  Task 05 - Model Evaluation: Test Dataset

In this task, you are required your fine tuned model on the Test dataset using ``evaluator.py`` and compare your results with random and 1-baseline.

In [ ]:
#Write your code
# switchig model to evaluation mode
LLM.eval()
predictions = []
# Process each text
test_args = datasets['training']['arguments']['Premise'].values.tolist()
for text in text_list:
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs = inputs.to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.sigmoid(logits)  # Apply sigmoid to convert logits to probabilities

    predictions.append(probabilities)


# Define the new threshold (e.g., 0.4)
new_threshold = 0.5

# Create an empty DataFrame with the correct columns
df_submission = pd.DataFrame(columns=label_columns)

# Iterate through the rows of all_probabilities_array
for row in predictions:
    # Create a mask for values greater than or equal to the new threshold
    mask = row >= new_threshold

    # Check if any value is above the threshold
    if mask.any():
        # If there is at least one value above the threshold, use the mask
        thresholded_row = mask.astype(int)
    else:
        # If there are no values above the threshold, take the top 1 value
        top_1_index = np.argmax(row)
        thresholded_row = np.zeros(len(label_columns), dtype=int)
        thresholded_row[top_1_index] = 1

    # Reshape the thresholded_row to match the length of the index
    thresholded_row = thresholded_row.reshape(1, -1)

    # Append the thresholded row to the DataFrame
    df_submission = df_submission.append(pd.DataFrame(thresholded_row, columns=label_columns), ignore_index=True)

# Convert the DataFrame to integer type
df_submission = df_submission.astype(int)

## [10 Points]  Task 06 - Model Evaluation: Zhihu Dataset

In this task, you are required your fine tuned model on the Zhihu
 dataset using ``evaluator.py`` and compare your results with random and 1-baseline.

## [10 Points]  Task 07 - Model Evaluation: Nahjalbalagha Dataset

In this task, you are required your fine tuned model on the Nahjalbalagha dataset using ``evaluator.py`` and compare your results with random and 1-baseline.

# References

In this section, cite any resources or references that you use for solving this assignment.
